In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
def new_positions_spherical_coordinates(num_points, radius, x_offset=0.0, y_offset=0.0, z_offset=0.0, stddev=0.0):
    radius = radius + stddev * np.random.rand()
    print(radius)
    theta = np.random.uniform(0.,2. * np.pi ,(num_points,1))
    phi = np.arccos(1-2*np.random.uniform(0.0,1.,(num_points,1)))
    x = radius * np.sin( theta ) * np.cos( phi ) + x_offset + stddev * np.random.rand()
    y = radius * np.sin( theta ) * np.sin( phi ) + y_offset + stddev * np.random.rand()
    z = radius * np.cos( theta ) + z_offset + stddev * np.random.rand()
    return (x,y,z)

In [3]:
np.set_printoptions(precision=4)

In [4]:
# Generate random point set
r = 44.0
n = 1000
(x,y,z) = new_positions_spherical_coordinates(n, r,
                                              x_offset=1.0,
                                              y_offset=2.0,
                                              z_offset=3.0, 
                                              stddev=0.1)

44.0968072721


In [5]:
# Point set from Hildenbrand
# n = 5
# x = [1.0, 1.0, 0.0, 0.0, -1.0]
# y = [0.0, 1.0, 0.0, 1.0,  0.0]
# z = [0.0, 0.0, 1.0, 1.0,  1.0]

In [6]:
ps = [np.array(np.ravel(p).tolist() + 
               [1.0, 0.5*np.linalg.norm(p)**2]).reshape(5,1) 
      for p in zip(x,y,z)]

## Total Least Squares Fitting of $k$-Spheres in $n$-D Euclidean Space Using an $(n + 2)$-D Isometric Representation

Leo Dorst 2014

Journal of Mathematical Imaging and Vision

http://link.springer.com/article/10.1007%2Fs10851-014-0495-2


Cost function:
$$\frac{1}{N}\sum_i\frac{(p_i \cdot x)^2}{x^2}$$

In [7]:
M = np.zeros((5,5))
M[:3,:3] = np.eye(3)
M[3,4] = -1
M[4,3] = -1
P = np.zeros((5,5))
for p in ps:
    P += np.inner(np.outer(p,p),M)
(w,v) = np.linalg.eig(P / float(len(ps)))
idx = np.where(w == np.min(w[w>0]))[0][0]
x_ = v[:,idx]
x_ /= x_[3]
print(x_)
radius = np.sqrt(x_[0]*x_[0] + x_[1]*x_[1] + x_[2]*x_[2] - 2*x_[4])
print("Estimated radius: {}".format(radius))

[   1.0837    2.0029    3.0921    1.     -964.8907]
Estimated radius: 44.0968072721


In [8]:
# M

In [9]:
# P

## Foundations of Geometric Algebra Computing

Dietmar Hildenbrand 2013

In [10]:
P = np.zeros((5,5))
for p in ps:
    P += np.outer(p,p)
(w,v) = np.linalg.eig(P)
idx = np.where(w == np.min(w[w>0]))[0][0]
x_ = v[:,idx]
x_ /= x_[4]
print(x_)
radius = np.sqrt(x_[0]*x_[0] + x_[1]*x_[1] + x_[2]*x_[2] - 2*x_[3])
print("Estimated radius: {}".format(radius))

[-680.1557  -14.3937   -8.8803    0.8233    1.    ]
Estimated radius: 680.364696223


In [11]:
# P

In [12]:
import sys
sys.path.append('../build/Debug/')
import libsphere_fit as sphere_fit

In [19]:
sf = sphere_fit.SphereFit()

solver_options = {'trust_region_strategy_type':'LEVENBERG_MARQUARDT',
                  'linear_solver_type':'DENSE_QR',
                  'max_num_iterations': 25, 
                  'num_threads': 12,
                  'num_linear_solver_threads':12,
                  'parameter_tolerance': 10e-12,
                  'function_tolerance': 10e-12}
sf.set_solver_options(solver_options)


sphere = np.array([1.0, 1.0, 1.0, 1.0, 1.0]).reshape(5,1)
points = np.ascontiguousarray(np.transpose([x,y,z])).reshape(n,3).copy()
x_ = sf.run(sphere, points).copy()
x_ = x_ / x_[3]
print(x_.reshape(-1))
radius = np.sqrt(x_[0]*x_[0] + x_[1]*x_[1] + x_[2]*x_[2] - 2*x_[4])[0]
print("Estimated radius: {}".format(radius))

[   1.0837    2.0029    3.0921    1.     -964.8907]
Estimated radius: 44.0968072721


In [20]:
sphere.reshape(-1)

array([  0.0487,   0.09  ,   0.139 ,   0.0449, -43.3646])

In [21]:
sf.summary()['linear_solver_type_used']

'DENSE_QR'

In [22]:
sf.summary()

{'brief_report': 'Ceres Solver Report: Iterations: 8, Initial cost: 4.748744e+08, Final cost: 3.383180e-21, Termination: CONVERGENCE',
 'iterations': [{'cost': 474874439.4890291,
   'cost_change': 0.0,
   'eta': 0.1,
   'gradient_max_norm': 1905672165.712697,
   'iteration': 0,
   'linear_solver_iterations': 0,
   'relative_decrease': 0.0,
   'step_norm': 0.0,
   'trust_region_radius': 10000.0},
  {'cost': 162463339.8635205,
   'cost_change': 312411099.62550855,
   'eta': 0.0,
   'gradient_max_norm': 342908970.87357825,
   'iteration': 1,
   'linear_solver_iterations': 1,
   'relative_decrease': 0.6578905455561771,
   'step_norm': 0.7210447956877726,
   'trust_region_radius': 10325.127563120355},
  {'cost': 55317326.347779915,
   'cost_change': 107146013.51574059,
   'eta': 0.0,
   'gradient_max_norm': 74609661.90800117,
   'iteration': 2,
   'linear_solver_iterations': 1,
   'relative_decrease': 0.6595125299638808,
   'step_norm': 2.855379761792053,
   'trust_region_radius': 10671.629

In [17]:
# from mpl_toolkits.mplot3d import Axes3D
# fig = plt.figure()
# ax = fig.gca(projection='3d')
# ax.set_aspect("equal")
# u, v = np.mgrid[0:2*np.pi:20j, 0:-np.pi:10j]
# sx= r * np.cos(u)*np.sin(v)
# sy= r * np.sin(u)*np.sin(v)
# sz= r * np.cos(v)
# ax.plot_wireframe(sx, sy, sz, color="r")

# # ax.scatter3D(x,y,z)
# sx= radius * np.cos(u)*np.sin(v) + x_[0]
# sy= radius * np.sin(u)*np.sin(v) + x_[1]
# sz= radius * np.cos(v) + x_[2]
# ax.plot_wireframe(sx, sy, sz, color="b")